In [3]:
from Crypto.Util.number import *

In [4]:
FLAG = b'FAKE{XXXXXXXXXXXXXXXXXXXXXX}'

N = 7
WORD_SIZE = 32
WORD_MASK = (1 << WORD_SIZE) - 1

In [5]:
state = [1927245640, 871031439, 789877080, 4042398809, 3950816575, 2366948739, 935819524]

In [6]:
def encrypt(m):
    state = [int.from_bytes(m[i:i+4]) for i in range(0, len(m), 4)]
    assert len(state) == N

    def xor_shift():
        nonlocal state
        t = state[0] ^ ((state[0] << 11) & WORD_MASK)
        for i in range(N-1):
            state[i] = state[i+1]
        state[-1] = (state[-1] ^ (state[-1] >> 19)) ^ (t ^ (t >> 8))

    for _ in range(M):
        xor_shift()

    return state

In [7]:
def gen_left_shift_mat(dimension):
	nn = [[0 for _ in range(dimension)] for _ in range(dimension)]
	for i in range(dimension - 1):
		nn[i][i+1] = 1
	return Matrix(GF(2), nn)

def gen_right_shift_mat(dimension):
	Un = gen_left_shift_mat(dimension)
	return Un.transpose()

def int_2_vec(si, nbits):
	return vector(GF(2), list(map(int, bin(si)[2:].zfill(nbits))))

def gen_svs_mat(sis):
	# concat list of vecs
	svs = []
	for i in sis:
		svs.extend(list(i))
	return vector(GF(2), svs)
    
def state_2_ints(state_vec, dimension):
	# create state_vec to individual integer in state
	ss = []
	state_vec = list(state_vec)
	for i in range(0, len(state_vec), dimension):
		val = int(''.join(map(str, state_vec[i: i+dimension])), 2)
		ss.append(val)
	return ss

def ints_2_state(state_ints, dimension):
	init_s = []
	for i in state_ints:
		init_s.append(int_2_vec(i, dimension))
	return gen_svs_mat(init_s)

In [8]:
    def xor_shift(y,count):
        state=y.copy()
        for i in range(count):
            t = state[0] ^^ ((state[0] << 11) & WORD_MASK)
            for i in range(7-1):
                state[i] = state[i+1]
            state[-1] = (state[-1] ^^ (state[-1] >> 19)) ^^ (t ^^ (t >> 8))
        return state

In [9]:
def gmm(dimension):
    t1 = [N,N,A]
    t1+=[N for i in range(dimension-3)]
    block = [[N for _ in range(dimension)] for _ in range(dimension-2)]
    for i in range(6):
        block[i][2+i] = I
    t2 = [Z]+[N for i in range(dimension-2)]+[Y]
    block = [t1]+block+[t2]
    return block_matrix(block)

In [10]:
dimension = 32
Un = gen_left_shift_mat(dimension)
Ln = gen_right_shift_mat(dimension)
I = Matrix.identity(GF(2),32)
N = Matrix(GF(2),32)

In [11]:
### encrytion proof
A = Un^11+I
Y = Ln^19+I
Z = Ln^8 + I

In [12]:
b=gmm(8)

In [13]:
t = state[0] ^^ ((state[0] << 11) & WORD_MASK)

In [14]:
s1 = [t]+state

In [15]:
b1=b^5*ints_2_state(s1,32)

In [16]:
a1=xor_shift(state,5)

In [17]:
a2=state_2_ints(b1,32)

In [18]:
a1 ==a2[1:]

True

In [19]:
def gmm2(dimension):
    block = [[N for _ in range(dimension)] for _ in range(dimension-1)]
    for i in range(6):
        block[i][i] = I
    t2 = [[N for i in range(dimension-2)] + [N,A^-1*Z^-1]]
    t2 = t2 + block
    return block_matrix(t2)

In [20]:
def gmm3(dimension):
    block = [[N for _ in range(dimension)] for _ in range(dimension-1)]
    for i in range(6):
        block[i][i] = I
    t2 = [[N for i in range(dimension-2)] + [-Y,I]]
    block+=t2
    return block_matrix(block)

In [21]:
p1 = gmm2(7)

In [22]:
h1 = gmm3(7)

In [23]:
b2= (p1*h1)^(17005450388330379)*ints_2_state(state,32)

In [24]:
x1=state_2_ints(b2,32)

In [25]:
t1=b''
for i in x1:
    t1+= long_to_bytes(i)

In [26]:
t1

b'FLAG{m47r1x_!n_8inary_w0rld}'